In [ ]:
import json
import os
import re
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# function for writing dataset

In [ ]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

In [ ]:
for each in os.listdir(corpus_path):
  print(each)

ansarian.xml
ayati.xml
bahrampour.xml
fooladvand.xml
ghomshei.xml
khorramdel.xml
khorramshahi.xml
makarem.xml
moezzi.xml
mojtabavi.xml
sadeqi.xml


for each question, replace the context with other translations

In [ ]:
with open('/content/drive/MyDrive/single_verse.jsonl', 'r') as f:
  dict_data_list = []
  for line in f:
   data = json.loads(line)
   dict_data_list.append(data)

corpus_path = '/content/drive/MyDrive/fa'

new_data_list = []
exceptions_list = []
multiple_occurence = []

for each in os.listdir(corpus_path):
  if each != 'makarem.xml':
    file_path = corpus_path + '/' + each
    with open(file_path, 'r', encoding="utf8") as f:
      content = f.read()
    soup= BeautifulSoup(content, 'xml')
    for qa in dict_data_list:
      question = qa['question']
      answer = qa['answers']
      verse_id = qa['pq_id']

      try:
        translated_verse = soup.find(id=verse_id).contents[0]
        translated_verse = re.sub("[\(\[].*?[\)\]]", "", translated_verse)
      except:
        exceptional_data = {}
        exceptional_data['verse_id'] = verse_id
        exceptional_data['translation_file'] = each
        exceptions_list.append(exceptional_data)

      if answer in translated_verse:
        new_data = {}
        new_data['context'] = translated_verse
        new_data['pq_id'] = qa['pq_id']
        new_data['question'] = question
        new_data['translation_file'] = each

        if translated_verse.count(answer) == 1:
          start_char = translated_verse.find(answer)
          answers_list = []
          answer_dict = {}
          answer_dict['text'] = answer
          answer_dict['start_char'] = start_char
          answers_list.append(answer_dict)
          new_data['answers'] = answers_list
          new_data_list.append(new_data)

        else:
          new_data['answers'] = answer
          new_data['freq'] = translated_verse.count(answer)
          multiple_occurence.append(new_data)

      else:
        continue



save new augmented dataset

In [ ]:
output_path = '/content/drive/MyDrive/extra_translations.jsonl'
dump_jsonl(new_data_list, output_path)

Wrote 5632 records to /content/drive/MyDrive/extra_translations.jsonl


In [ ]:
exceptional_data

{'verse_id': 's58.10', 'translation_file': 'sadeqi.xml'}

In [ ]:
new_data_list

[{'context': 'در آنها میوه های فراوان و درخت خرما و انار است.',
  'pq_id': 's55.68',
  'question': 'دو نوع از میوه\u200c\u200cهای درختان بهشتی را نام ببرید؟',
  'translation_file': 'ansarian.xml',
  'answers': 'خرما و انار'},
 {'context': 'در آن دو، ميوه و خرما و انار است',
  'pq_id': 's55.68',
  'question': 'دو نوع از میوه\u200c\u200cهای درختان بهشتی را نام ببرید؟',
  'translation_file': 'bahrampour.xml',
  'answers': 'خرما و انار'},
 {'context': 'در آن دو، ميوه و خرما و انار است.',
  'pq_id': 's55.68',
  'question': 'دو نوع از میوه\u200c\u200cهای درختان بهشتی را نام ببرید؟',
  'translation_file': 'fooladvand.xml',
  'answers': 'خرما و انار'},
 {'context': 'در آن دو بهشت نیز هرگونه میوه خوش و خرما و انار بسیار است.',
  'pq_id': 's55.68',
  'question': 'دو نوع از میوه\u200c\u200cهای درختان بهشتی را نام ببرید؟',
  'translation_file': 'ghomshei.xml',
  'answers': 'خرما و انار'},
 {'context': 'در آن باغها انواع میوه\u200cها و از جمله خرما و انار است. [[«نَخْلٌ»: درخت خرما. ذکر درخت خرما ب